# Top of Book Snapshots at One-Minute Intervals

This notebook demonstrates how to capture best bid and ask prices at regular intervals from ITCH 5.0 data.

Top of book snapshots are useful for:
- Creating time series of bid-ask spreads
- Analyzing price movements over time
- Calculating mid-price evolution

This example uses the `S081321-v50-AAPL-SPY.itch50.gz` file generated in the [Extracting Symbols](02_extracting_symbols.ipynb) example.

In [ ]:
from pathlib import Path
import datetime
import pandas as pd
from meatpy.itch50 import ITCH50MessageReader, ITCH50MarketProcessor
from meatpy.event_handlers.lob_recorder import LOBRecorder
from meatpy.writers.parquet_writer import ParquetWriter

# Define paths and parameters
data_dir = Path("data")

file_path = data_dir / "S081321-v50-AAPL-SPY.itch50.gz"
outfile_path = data_dir / "spy_tob.parquet"
book_date = datetime.datetime(2021, 8, 13)


with ITCH50MessageReader(file_path) as reader, ParquetWriter(outfile_path) as writer:
    processor = ITCH50MarketProcessor("SPY", book_date)

    # We only care about the top of book
    tob_recorder = LOBRecorder(writer=writer, max_depth=1)
    # Generate a list of timedeltas from 9:30 to 16:00 (inclusive) in 1-minute increments
    market_open = book_date + datetime.timedelta(hours=9, minutes=30)
    market_close = book_date + datetime.timedelta(hours=16, minutes=0)
    record_timestamps = [market_open + datetime.timedelta(minutes=i) 
                     for i in range(int((market_close - market_open).total_seconds() // 60) + 1)]
    tob_recorder.record_timestamps = record_timestamps

    # Attach the recorders to the processor
    processor.handlers.append(tob_recorder)

    for message in reader:
        processor.process_message(message)

In [2]:
pd.read_parquet(outfile_path).head(10)

,Timestamp,Type,Level,Price,Volume,N Orders
0,2021-08-13 09:30:00,Ask,1,445.22,300,3
1,2021-08-13 09:30:00,Bid,1,445.68,4200,4
2,2021-08-13 09:31:00,Ask,1,445.22,300,3
3,2021-08-13 09:31:00,Bid,1,445.68,4200,4
4,2021-08-13 09:32:00,Ask,1,445.22,300,3
5,2021-08-13 09:32:00,Bid,1,445.68,4200,4
6,2021-08-13 09:33:00,Ask,1,445.22,300,3
7,2021-08-13 09:33:00,Bid,1,445.68,4200,4
8,2021-08-13 09:34:00,Ask,1,445.22,300,3
9,2021-08-13 09:34:00,Bid,1,445.68,4200,4
